In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image, ImageTk
from faker import Faker
from transformers import pipeline
from keybert import KeyBERT

In [2]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

# Basic Dataset erstellen

3 Kategorien: Hotels, Restaurants & Aktivitäten

In [3]:
df_rest = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
df_act = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
df_hotel = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

C:\Users\michi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3203: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,199,200,201,202,203,204,211,215,235,236,237,238,564,565,566,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,631,634,635,636,637,639,640) have m

Restaurants (Shape):  (20639, 643)
Activities (Shape):  (36376, 509)
Hotels (Shape):  (32038, 323)


In [4]:
columns_to_keep = [
    "stars",
    "publishedAtDate",
    "name",
    "text",
    "title",
    "subTitle",
    "description",
    "price",
    "totalScore",
    "likesCount",
    "isLocalGuide",
    "reviewId",
    "categoryName",
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9",
    "reviewUrl",
    "reviewerId",
    "reviewerNumberOfReviews",
    "reviewerPhotoUrl",
    "reviewerUrl",
    "reviewsCount",
    "scrapedAt",
    "state",
    "street",
    "temporarilyClosed",
    "url",
    "website",

    # at least for restaurants
    "categories/0",
    "categories/1",
    "categories/2",
    "categoryName"
]

In [5]:
df_rest = df_rest.dropna(subset=["reviewImageUrls/0"])
df_rest = df_rest[columns_to_keep]
df_rest['genre'] = 'restaurant'
print(df_rest.shape)

(1179, 40)


In [6]:
df_act = df_act.dropna(subset=["reviewImageUrls/0"])
df_act = df_act[columns_to_keep]
df_act['genre'] = 'activity'
print(df_act.shape)

(3844, 40)


In [7]:
df_hotel = df_hotel.dropna(subset=["reviewImageUrls/0"])
df_hotel = df_hotel[columns_to_keep]
df_hotel['genre'] = 'hotel'
print(df_hotel.shape)

(1846, 40)


In [12]:
df = pd.concat([df_rest, df_act, df_hotel], ignore_index=True)
print(df.shape)
df.to_csv('all_rest_act_hotel.csv', index=False)

(6869, 40)


Resizing the images to 256-256 pixels via gettin the better url

In [14]:
image_urls_cols = [
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9"
]

for col in image_urls_cols:
    df[col] = df[col].str.replace("=w150-h150-k-no-p", "=w256-h256-p-k-no")

print(df.shape)

(6869, 40)


### Extract Keywords With KeyBert

In [17]:
df['text'] = df['text'].astype(str)
print("Convertion done.")

df['keywords'] = df['text'].apply(lambda x: kw_model.extract_keywords(x,keyphrase_ngram_range=(1,1) ,
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5))
print("Keywords extraction done.")
df.to_csv('base_keywords_sentiment.csv', index=False)


Convertion done.


### Sentiment analyse

In [15]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [234]:
df['sentiment'] = df['text'].apply(lambda x: sentiment_task(x)[0]['label'])
print("done with sentiment label. ")
df['sentiment_score'] = df['text'].apply(lambda x: sentiment_task(x)[0]['score'])
print("done with sentiment score ")

# df['sent_score_1'] = df['text'].apply(lambda x: sentiment_task(x)[1]['score'])
# df['sent_score_2'] = df['text'].apply(lambda x: sentiment_task(x)[2]['score'])

# sentiment_labels = []
# sent_score_0 = []
# sent_score_1 = []
# sent_score_2 = []

# for idx, row in df.iterrows():
#     # Print the row index
#     print("Processing row ", idx)

#     sentiment = sentiment_task(row['text'])
    
#     # Extract the sentiment label and scores and append them to the corresponding lists
#     sentiment_labels.append(sentiment[0]['label'])
#     sent_score_0.append(sentiment[0]['score'])

#     print(sentiment[0]['label'])
#     print(sentiment[0]['score'])

# df['sentiment'] = sentiment_labels
# df['sent_score_0'] = sent_score_0
# df['sent_score_1'] = sent_score_1
# df['sent_score_2'] = sent_score_2

In [ ]:
print(df_keywords_test['keywords'])

#only store the keywords in a list not the scores

df_keywords_test['keywords_only'] = df_keywords_test['keywords'].apply(lambda x: [i[0] for i in x])
print(df_keywords_test['keywords_only'])

## Gradio Anwendung Versuch

### HTML Bewertung im Stil von Google Maps erstellen

In [ ]:
def generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images):
    full_stars = int(stars)
    half_star = (stars % 1) > 0
    empty_stars = 5 - full_stars - half_star
    full_star_html = '<span class="star full">&#9733;</span>'
    half_star_html = '<span class="star half">&#9733;</span>'
    empty_star_html = '<span class="star empty">&#9733;</span>'
    stars_html = (full_star_html * full_stars) + (half_star_html * half_star) + (empty_star_html * empty_stars)
    user_name = user if not is_local_guide else f"{user} (local guide)"
    
    html = f"""
    <style>
      .star.full {{
        color: #f1c40f;
      }}
      .star.half {{
        color: #f1c40f;
        position: relative;
      }}
      .star.half::before {{
        content: '\\2605';
        position: absolute;
        left: 0;
        color: #ddd;
      }}
      .star.empty {{
        color: #ddd;
      }}
    </style>
    <div class="review">
      <div class="user">
        <img src={user_image} alt="User profile picture">
        <div class="user-info">
          <h4>{user_name}</h4>
          <p>Submitted on {date}</p>
        </div>
      </div>
      <div class="rating">
        {stars_html}
      </div>
      <div class="review-text">
        <p>{text}</p>
      </div>
      <div class="review-image">
        <img src={review_images} alt="Restaurant image">
      </div>
    </div>
    """
    return html

In [ ]:
def generate_deepfake(review_about, stars):
    text = fake.text()
    stars = fake.random_int(min=1, max=5, step=1)
    date = fake.date_between(start_date='-1y', end_date='today')
    user = fake.name()
    local_guide = fake.boolean(chance_of_getting_true=7)
    html_review = generate_review_html(stars, date, user, local_guide, text, user_image="https://via.placeholder.com/50/%22%20alt=/%22Restaurant%20image/", review_images = "https://lh5.googleusercontent.com/p/AF1QipPsYMpfUpwmH5ltkPNIeDbgPivvBstyJCea32vJ=w150-h150-k-no-p")
    return html_review

def verify_review(stars, date, user, local_guide, text, image):
    return "FAKE", "0.9"


In [ ]:
with gr.Blocks() as demo:
    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake, inputs=[review_about, review_stars], outputs=[html_output])
    
    with gr.Tab ("Verify Review"):
        with gr.Row(): 
            input_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
            input_date = gr.Textbox(label="Date", default="2023-01-01")
            input_user = gr.Textbox(label="User")
            input_local_guide = gr.Checkbox(label="Local Guide")
       
        input_text = gr.Textbox(label="Review Text")
        input_image = gr.Image(label="Review Image")
        verify_btn = gr.Button("Verify Review")
        output_text = gr.Textbox(label="Generated Text")
        output_score = gr.Textbox(label="Score")
        verify_btn.click(fn=verify_review, inputs=[input_stars, input_date, input_user, input_local_guide, input_text, input_image], outputs=[output_text, output_score])

demo.launch(debug=True)